Check how often a column in the DeSSI test set has a value that also appears in training data for the same class   
  
Note: Even if no value of test data appears in train data the model can learn the prediction if the column names are quite similar, this is not analyzer here

In [5]:
import pandas as pd
import plotly.graph_objects as go

# DeSSI

In [10]:
def convert_dataframes(df, df_labels):
    df_labels = df_labels.T
    df_labels.columns = df.columns
    df = pd.concat([df, df_labels], ignore_index=True)
    return df

def load_data(path, labels=True):
    if labels == True:
        add = "labels"
        val = "dev"
    else:
        add = "classes"
        val = "validation"
    train = pd.read_csv(path + 'train.csv')
    train_labels = pd.read_csv(path + f'train_{add}.csv')
    dev = pd.read_csv(path + f'{val}.csv')
    dev_labels = pd.read_csv(path + f'{val}_{add}.csv')
    test = pd.read_csv(path + 'test.csv')
    test_labels = pd.read_csv(path + f'test_{add}.csv')
    train = convert_dataframes(train, train_labels)
    dev = convert_dataframes(dev, dev_labels)
    test = convert_dataframes(test, test_labels)
    return train, dev, test

In [11]:
train_dessi, dev_dessi, test_dessi = load_data('../../dessi/DeSSI_v2/')

C:\Users\Luca\AppData\Local\Temp\ipykernel_17944\685814217.py:14: DtypeWarning: Columns (62,107,161,241,255,324,449,503,686,720,807,812,889,934,1073,1113,1133,1154,1158,1242,1317,1333,1394,1408,1470,1553,1577,1674,1729,1762,1872,1913,1940,2045,2131,2139,2209,2288,2404,2436,2601,2627,2663,2779,2841,2960,2994,3061,3063,3080,3129,3136,3250,3319,3439,3464,3499,3520,3596,3637,3649,3748,3778,3787,3808,3902,3907,3917,3951,4009,4034,4037,4069,4129,4210,4218,4252,4325,4359,4386,4479,4646,4649,4652,4710,4936,4939,5013,5065,5069,5348,5352,5356,5399,5476,5702,5736,5889,5962,6072,6094,6107,6140,6159,6337,6360,6383,6407,6415,6423,6507,6555,6567,6590,6593,6689,6809,6815,6920,6995,7042,7127,7163,7229,7295,7301,7319,7321,7354,7399,7403,7465,7474,7593,7594,7694,7719,7798,7872,7979,8179,8298,8308,8473,8542,8565,8568,8570,8579,8633,8662,8707,8835,8846,8859,8899,8933,9024,9084,9126,9170,9184,9198,9251,9329,9412,9505,9588,9592,9610,9652,9803,9818,9907,9908,9952,9975,10073,10099,10136,10161,10359,10369,10407

In [12]:
train_dev_dessi = train_dessi.merge(dev_dessi, left_index=True, right_index=True)

In [13]:
sol = dict()
for label in train_dev_dessi.loc[100].unique():
    s = train_dev_dessi.loc[100] == label
    train_dev = train_dev_dessi.iloc[:100,:]
    train_data = set(train_dev[s.index[s==True].tolist()].values.flatten()) 
    s2 = test_dessi.loc[100] == label
    
    has_value = []
    test = test_dessi.iloc[:100,:]
    for c in s2.index[s2==True].tolist():
        has_value.append(any(val in train_data for val in test[c].values))
    sol[label] = dict(pd.Series(has_value, dtype="object").value_counts(normalize=True))

In [14]:
pop_keys = []
for key, val in sol.items():
    if val == dict():
        pop_keys.append(key)
for key in pop_keys:
    sol.pop(key)
sol = {key: sol[key] for key in sorted(sol.keys(), reverse=True)}

In [15]:
fig = go.Figure()
fig.add_trace(go.Bar(y=list(sol.keys()), x=[a.get(False) for a in sol.values()], name="does not contain<br>element from train",
                      orientation='h', marker=dict(color="green")))
fig.add_trace(go.Bar(y=list(sol.keys()), x=[a.get(True) for a in sol.values()], name="contains element<br>from train", orientation='h', marker=dict(color="red")))
fig.update_layout(barmode='stack', title="Percentage of columns in DeSSI's test data per class that contain a value<br>that also appears in the training data for the same class", 
                  width=1000, height=800)
fig.update_xaxes(title="Classes")
fig.update_yaxes(title="Percentage")
fig.show()